In [1]:
import gym, ptan, torch
import torch.optim as optim
from tensorboardX import SummaryWriter
from lib import dqn_model, common


In [ ]:
if __name__ == "__main__":
    params = common.HYPERPARAMS['pong']
    device = torch.device("cuda")
    env = gym.make(params['env_name'])
    env = ptan.common.wrappers.wrap_dqn(env)
    writer = SummaryWriter(comment="-" + params['run_name'] + "-basic")
    net = dqn_model.DQN(env.observation_space.shape ,env.action_space.n).to(device)
    tgt_net = ptan.agent.TargetNet(net)
    selector = ptan.actions.EpsilonGreedyActionSelector(epsilon = params['epsilon_start'])
    epsilon_tracker = common.EpsilonTracker(selector, params)
    agent = ptan.agent.DQNAgent(net, selector,device=device)
    exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent,gamma=params['gamma'], steps_count=1)
    buffer = ptan.experience.ExperienceReplayBuffer(exp_source, buffer_size= params['replay_size'])
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    frame_idx = 0
    with common.RewardTracker(writer, params['stop_reward']) as reward_tracker:
        while True:
            env.render()
            frame_idx += 1
            buffer.populate(1)
            epsilon_tracker.frame(frame_idx)
            new_rewards = exp_source.pop_total_rewards()
            if new_rewards:
                if reward_tracker.reward(new_rewards[0], frame_idx, selector.epsilon):
                    break
            if len(buffer) < params['replay_initial']:
                continue
            optimizer.zero_grad()
            batch = buffer.sample(params['batch_size'])
            loss_v = common.calc_loss_dqn(batch,net, tgt_net.target_model, gamma = params['gamma'], device=device)
            loss_v.backward()
            optimizer.step()
            if frame_idx % params['target_net_sync'] == 0:
                tgt_net.sync()
                
            

786: done 1 games, mean reward -21.000, speed 84.29 f/s, eps 0.99
1605: done 2 games, mean reward -21.000, speed 119.79 f/s, eps 0.98
2536: done 3 games, mean reward -20.333, speed 122.05 f/s, eps 0.97
3375: done 4 games, mean reward -20.500, speed 120.28 f/s, eps 0.97
4270: done 5 games, mean reward -20.600, speed 115.37 f/s, eps 0.96
5298: done 6 games, mean reward -20.500, speed 122.95 f/s, eps 0.95
6170: done 7 games, mean reward -20.571, speed 100.40 f/s, eps 0.94
7180: done 8 games, mean reward -20.500, speed 78.63 f/s, eps 0.93
8035: done 9 games, mean reward -20.556, speed 82.86 f/s, eps 0.92
8935: done 10 games, mean reward -20.500, speed 85.92 f/s, eps 0.91
9756: done 11 games, mean reward -20.545, speed 85.52 f/s, eps 0.90
10533: done 12 games, mean reward -20.583, speed 36.71 f/s, eps 0.89
11397: done 13 games, mean reward -20.615, speed 23.75 f/s, eps 0.89
12214: done 14 games, mean reward -20.643, speed 25.87 f/s, eps 0.88
13060: done 15 games, mean reward -20.667, speed 